# 범주형 데이터 검정

* 독립성 검정 : 두 변수가 있을 때 두 변수가 서로 독립인지 아닌지에 대해 검정
* 적합도 검정 : 한 변수의 표본분포가 어떤 이론분포와 일치하는지를 검정


카이제곱분포가 사용되기 때문이 카이제곱 검정이라고도 한다.

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

%precision 3

'%.3f'

### 1. 독립성 검정 - 카이제곱검정

광고의 종류와 상품의 구입 여부가 독립이라면 A광고든 B광고든 상관없이 구입 비율에 변화가 없을 것이다. 그러나 광고종류와 구입여부가 독립이 아니라면 상품 구입 비율에 유의한 차이가 있을 것이다. 이를 검정하는 것이 독립성 검정이다.


In [2]:
ad_df = pd.read_csv('./data/ch11_ad.csv')
n = len(ad_df)
print(n)
ad_df.head()

1000


,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다


#### 분할표

pandas 의 crosstab을 사용하면 간단하게 분할표를 생성할 수 있다. 

In [3]:
ad_cross = pd.crosstab(ad_df['광고'], ad_df['구입'])
ad_cross

구입,하지 않았다,했다
광고,,
A,351,49
B,549,51


분할표를 기초로 광고 A와 광고 B에서의 상품 구입 비율을 구한다.

In [4]:
ad_cross['했다'] / (ad_cross['했다'] + ad_cross['하지 않았다'])

광고
A    0.1225
B    0.0850
dtype: float64

In [5]:
# 구입하지 않은 사람과 구입한 사람
n_not, n_yes = ad_cross.sum()
n_not, n_yes

(900, 100)

In [6]:
# A광고를 본 사람과 B광고를 본 사람
n_adA, n_adB = ad_cross.sum(axis=1)
n_adA, n_adB

(400, 600)

#### 관측도수와 기대도수
광고와 구입이 서로 독립일때 기대되는 도수를 기대도수라고 한다.  
실제로 관측된 데이터의 도수를 관측도수라고 한다. 

기대도수를 구하면 다음과 같다.

In [7]:
ad_ef = pd.DataFrame({'했다': [n_adA * n_yes / n,
                              n_adB * n_yes / n],
                      '하지 않았다': [n_adA * n_not / n,
                                   n_adB * n_not / n]},
                      index=['A', 'B'])
ad_ef

,했다,하지 않았다
A,40.0,360.0
B,60.0,540.0


#### 카이제곱검정의 검정통계량

$$ \sum_{i}\sum_{j} \dfrac{(O_{ij} - E_{ij})^2}{E_{ij}}$$

이 검정통계량은 자유도가 (r-1)(c-1)인 카이제곱분포를 따른다. 이 경우 (r-1)(c-1) = 1이다.

In [8]:
y = ((ad_cross - ad_ef) ** 2 / ad_ef).sum().sum()
y

3.75

분포를 알고있으므로 p-value는 간단하게 cdf를 사용하여 계산할 수 있다. 

In [9]:
rv = stats.chi2(1)
1 - rv.cdf(y)

0.052807511416113395

p-value가 유의수준 0.05보다 크므로 귀무가설을 기각할 수 없다. 광고의 종류에 의한 구매여부의 차이가 인정되지 않는다. 즉 두 변수는 독립이다. 

### scipy.stats.chi2_countigency 함수

카이제곱 검정을 손쉽게 하기 위해 scipy.stats.chi2_countigency 함수를 사용한다.  
이 함수는 검정통계량, p-value, 자유도, 기대도수를 리턴한다.

In [10]:
chi2, p, dof, ef = stats.chi2_contingency(ad_cross,
                                          correction=False)
chi2, p, dof

(3.75, 0.052807511416113395, 1)

In [11]:
ef # 기대도수

array([[360.,  40.],
       [540.,  60.]])

# 실습

1. 다음과 같은 데이터가 있다. 버튼의 클릭률이 버튼색에 따라 달라지는지 검정해 보자. 

```
data = pd.DataFrame({'color':['blue', 'blue', 'red', 'red'],
                     'click':['click', 'not', 'click', 'not'],
                     'freq':[ 20, 230,  10,  40]})
data             
```

- pd.crosstab()을 사용하여 교차표를 만드시오.
- scipy.stats.chi2_contingency 함수를 사용하여 카이제곱 검정을 시행하시오.